In [63]:
# load CSV file
import os
import pandas as pd
df = pd.read_csv("data/train_10000.csv")


In [3]:
# review CSV data
df.head()

,MachineIdentifier,ProductName,EngineVersion,AppVersion,AvSigVersion,IsBeta,RtpStateBitfield,IsSxsPassiveMode,DefaultBrowsersIdentifier,AVProductStatesIdentifier,...,Census_FirmwareVersionIdentifier,Census_IsSecureBootEnabled,Census_IsWIMBootEnabled,Census_IsVirtualDevice,Census_IsTouchEnabled,Census_IsPenCapable,Census_IsAlwaysOnAlwaysConnectedCapable,Wdft_IsGamer,Wdft_RegionIdentifier,HasDetections
0,0000028988387b115f69f31a3bf04f09,win8defender,1.1.15100.1,4.18.1807.18075,1.273.1735.0,0,7,0,NaN,53447,...,36144.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,10.0,0.0
1,000007535c3f730efa9ea0b7ef1bd645,win8defender,1.1.14600.4,4.13.17134.1,1.263.48.0,0,7,0,NaN,53447,...,57858.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,8.0,0.0
2,000007905a28d863f6d0d597892cd692,win8defender,1.1.15100.1,4.18.1807.18075,1.273.1341.0,0,7,0,NaN,53447,...,52682.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,3.0,0.0
3,00000b11598a75ea8ba1beea8459149f,win8defender,1.1.15100.1,4.18.1807.18075,1.273.1527.0,0,7,0,NaN,53447,...,20050.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,3.0,1.0
4,000014a5f00daa18e76b81417eeb99fc,win8defender,1.1.15100.1,4.18.1807.18075,1.273.1379.0,0,7,0,NaN,53447,...,19844.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0


In [4]:
df.describe

<bound method NDFrame.describe of                    MachineIdentifier   ProductName EngineVersion  \
0   0000028988387b115f69f31a3bf04f09  win8defender   1.1.15100.1   
1   000007535c3f730efa9ea0b7ef1bd645  win8defender   1.1.14600.4   
2   000007905a28d863f6d0d597892cd692  win8defender   1.1.15100.1   
3   00000b11598a75ea8ba1beea8459149f  win8defender   1.1.15100.1   
4   000014a5f00daa18e76b81417eeb99fc  win8defender   1.1.15100.1   
5   000016191b897145d069102325cab760  win8defender   1.1.15100.1   
6   0000161e8abf8d8b89c5ab8787fd712b  win8defender   1.1.15100.1   
7   000019515bc8f95851aff6de873405e8  win8defender   1.1.15100.1   
8   00001a027a0ab970c408182df8484fce  win8defender   1.1.15200.1   
9   00001a18d69bb60bda9779408dcf02ac  win8defender   1.1.15100.1   
10  00001b3b3cf8c8c079a272ce51031753  win8defender   1.1.15100.1   
11  00001b924fcc6922321cfadbafd8a91a  win8defender   1.1.15100.1   
12  00001f26e9e5775277d6231fc6ac9e70  win8defender   1.1.15100.1   
13  000024872c

# upload to blob

In [108]:
# pip install azure-storage-blob
# pip install python-dotenv
from dotenv import load_dotenv

%load_ext dotenv
%dotenv

connect_str = os.environ['AZURE_STORAGE_CONNECTION_STRING']


The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [21]:
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient

blob_service_client = BlobServiceClient.from_connection_string(connect_str)
container_name = "bulk-insert"
container_client = blob_service_client.create_container(container_name)

In [82]:
local_path = "./data"
csv_file_name = "train_10000.csv"
# csv_file_name = "train_1000000.csv"
# csv_file_name = "train.csv"
upload_file_path = os.path.join(local_path, csv_file_name)


In [23]:
blob_client = blob_service_client.get_blob_client(container=container_name, blob=csv_file_name)

# Upload the created file
with open(upload_file_path, "rb") as data:
    blob_client.upload_blob(data)

In [72]:
# List the blobs in the container
blob_list = container_client.list_blobs()
for blob in blob_list:
    print("\t" + blob.name)

	train.csv
	train_10000.csv
	train_1000000.csv


# get SAS token

In [97]:
import logging
from datetime import datetime, timedelta
from azure.storage.blob import generate_blob_sas, BlobSasPermissions

AZURE_ACC_NAME = os.environ['AZURE_ACC_NAME']
AZURE_PRIMARY_KEY = os.environ['AZURE_PRIMARY_KEY']

In [115]:
def generate_sas_token(file_name):
    sas = generate_blob_sas(account_name=AZURE_ACC_NAME,
                            account_key=AZURE_PRIMARY_KEY,
                            container_name=container_name,
                            blob_name=file_name,
                            permission=BlobSasPermissions(read=True),
                            expiry=datetime.utcnow() + timedelta(hours=1))  # 2 hours expire for test

    sas_url ='https://'+AZURE_ACC_NAME+'.blob.core.windows.net/'+container_name+'/'+file_name+'?'+sas
    secret = sas
    return sas_url, secret

In [116]:
sas_url, secret = generate_sas_token(csv_file_name)

# bulk insert to azure sql database

## create table sql statement
```
create table malware(
MachineIdentifier varchar(256),
ProductName varchar(256),
EngineVersion varchar(256),
AppVersion varchar(256),
AvSigVersion varchar(256),
IsBeta int,
RtpStateBitfield float(24),
IsSxsPassiveMode int,
DefaultBrowsersIdentifier float(24),
AVProductStatesIdentifier float(24),
AVProductsInstalled float(24),
AVProductsEnabled float(24),
HasTpm int,
CountryIdentifier int,
CityIdentifier float(24),
OrganizationIdentifier float(24),
GeoNameIdentifier float(24),
LocaleEnglishNameIdentifier int,
Platform varchar(256),
Processor varchar(256),
OsVer varchar(256),
OsBuild int,
OsSuite int,
OsPlatformSubRelease varchar(256),
OsBuildLab varchar(256),
SkuEdition varchar(256),
IsProtected float(24),
AutoSampleOptIn int,
PuaMode varchar(256),
SMode float(24),
IeVerIdentifier float(24),
SmartScreen varchar(256),
Firewall float(24),
UacLuaenable float(24),
Census_MDC2FormFactor varchar(256),
Census_DeviceFamily varchar(256),
Census_OEMNameIdentifier float(24),
Census_OEMModelIdentifier float(24),
Census_ProcessorCoreCount float(24),
Census_ProcessorManufacturerIdentifier float(24),
Census_ProcessorModelIdentifier float(24),
Census_ProcessorClass varchar(256),
Census_PrimaryDiskTotalCapacity float(24),
Census_PrimaryDiskTypeName varchar(256),
Census_SystemVolumeTotalCapacity float(24),
Census_HasOpticalDiskDrive int,
Census_TotalPhysicalRAM float(24),
Census_ChassisTypeName varchar(256),
Census_InternalPrimaryDiagonalDisplaySizeInInches float(24),
Census_InternalPrimaryDisplayResolutionHorizontal float(24),
Census_InternalPrimaryDisplayResolutionVertical float(24),
Census_PowerPlatformRoleName varchar(256),
Census_InternalBatteryType varchar(256),
Census_InternalBatteryNumberOfCharges float(24),
Census_OSVersion varchar(256),
Census_OSArchitecture varchar(256),
Census_OSBranch varchar(256),
Census_OSBuildNumber int,
Census_OSBuildRevision bigint,
Census_OSEdition varchar(256),
Census_OSSkuName varchar(256),
Census_OSInstallTypeName varchar(256),
Census_OSInstallLanguageIdentifier float(24),
Census_OSUILocaleIdentifier int,
Census_OSWUAutoUpdateOptionsName varchar(256),
Census_IsPortableOperatingSystem int,
Census_GenuineStateName varchar(256),
Census_ActivationChannel varchar(256),
Census_IsFlightingInternal float(24),
Census_IsFlightsDisabled float(24),
Census_FlightRing varchar(256),
Census_ThresholdOptIn float(24),
Census_FirmwareManufacturerIdentifier float(24),
Census_FirmwareVersionIdentifier float(24),
Census_IsSecureBootEnabled int,
Census_IsWIMBootEnabled float(24),
Census_IsVirtualDevice float(24),
Census_IsTouchEnabled int,
Census_IsPenCapable int,
Census_IsAlwaysOnAlwaysConnectedCapable float(24),
Wdft_IsGamer float(24),
Wdft_RegionIdentifier float(24),
HasDetections int
)
```

In [117]:
# install pyodbc in Ubuntu : https://docs.microsoft.com/en-us/sql/connect/python/pyodbc/step-1-configure-development-environment-for-pyodbc-python-development?view=sql-server-ver15#linux
# pip install pyodbc
import pyodbc

server = os.environ['SQLSERVER']
database = os.environ['DATABASE']
username = os.environ['USERNAME']
password = os.environ['PASSWORD']
driver = '{ODBC Driver 17 for SQL Server}'
db_con_str = 'DRIVER='+driver+';SERVER=tcp:'+server+';PORT=1433;DATABASE='+database+';UID='+username+';PWD='+ password

with pyodbc.connect(db_con_str) as conn:
    with conn.cursor() as cursor:
        cursor.execute("SELECT TOP 3 name, collation_name FROM sys.databases") # test run
        row = cursor.fetchone()
        while row:
            print (str(row[0]) + " " + str(row[1]))
            row = cursor.fetchone()

master SQL_Latin1_General_CP1_CI_AS
malwaredb SQL_Latin1_General_CP1_CI_AS


In [118]:
# https://docs.microsoft.com/en-us/sql/relational-databases/import-export/examples-of-bulk-access-to-data-in-azure-blob-storage?view=sql-server-ver15

sec_cred = "scred_bulk_ins"
query = "CREATE DATABASE SCOPED CREDENTIAL " + sec_cred + " WITH IDENTITY = 'SHARED ACCESS SIGNATURE', SECRET = '" + secret + "';"
print(query)

# set "SCOPED CREDENTIAL"
with pyodbc.connect(db_con_str) as conn:
    with conn.cursor() as cursor:
        cursor.execute(query)

CREATE DATABASE SCOPED CREDENTIAL scred_bulk_ins WITH IDENTITY = 'SHARED ACCESS SIGNATURE', SECRET = 'se=2021-09-27T10%3A10%3A56Z&sp=rt&sv=2020-10-02&sr=b&sig=0btqr6SEBDqEi98Mk5fzYLs8BHDCHiqDKYYUVZXbA4k%3D';


## error issue about azure sql database master key
```
ProgrammingError: ('42000', '[42000] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]Please create a master key in the database or open the master key in the session before performing this operation. (15581) (SQLExecDirectW)')


https://docs.microsoft.com/en-us/sql/relational-databases/security/encryption/create-a-database-master-key?view=sql-server-ver15
CREATE MASTER KEY ENCRYPTION BY PASSWORD = 'YOUR-MASTER-KEY';
```

In [119]:
ext_ds = "malware_ds"
query = "CREATE EXTERNAL DATA SOURCE " + ext_ds + " WITH (TYPE = BLOB_STORAGE, LOCATION = 'https://" + AZURE_ACC_NAME + ".blob.core.windows.net', CREDENTIAL = " + sec_cred + ");"
print(query)

# set "EXTERNAL DATA SOURCE"
with pyodbc.connect(db_con_str) as conn:
    with conn.cursor() as cursor:
        cursor.execute(query)


CREATE EXTERNAL DATA SOURCE malware_ds WITH (TYPE = BLOB_STORAGE, LOCATION = 'https://dwstorstest.blob.core.windows.net', CREDENTIAL = scred_bulk_ins);


In [87]:
sql_table_name = "malware"
query = "BULK INSERT " + sql_table_name + " FROM '" + container_name + "/" + csv_file_name + "' WITH (DATA_SOURCE = '" + ext_ds + "', DATAFILETYPE = 'char', FIELDTERMINATOR = ',', ROWTERMINATOR = '0x0a', TABLOCK, CODEPAGE = 'UTF-8', FIRSTROW=2);"
print(query)

# run "BULK INSERT"
with pyodbc.connect(db_con_str) as conn:
    with conn.cursor() as cursor:
        cursor.execute(query)

BULK INSERT malware FROM 'bulk-insert/train.csv' WITH (DATA_SOURCE = 'malware_ds', DATAFILETYPE = 'char', FIELDTERMINATOR = ',', ROWTERMINATOR = '0x0a', TABLOCK, CODEPAGE = 'UTF-8', FIRSTROW=2);


In [1]:
# fetch dataset from ADX
# pip install azure-kusto-data

from azure.kusto.data import KustoClient, KustoConnectionStringBuilder
from azure.kusto.data.exceptions import KustoServiceError
from azure.kusto.data.helpers import dataframe_from_result_table
import pandas as pd

AAD_TENANT_ID = "<TenantId>"
KUSTO_CLUSTER = "https://help.kusto.windows.net/"
KUSTO_DATABASE = "Samples"

In [ ]:
KCSB = KustoConnectionStringBuilder.with_aad_device_authentication(KUSTO_CLUSTER)
KCSB.authority_id = AAD_TENANT_ID

In [ ]:
KUSTO_CLIENT = KustoClient(KCSB)
KUSTO_QUERY = "StormEvents | sort by StartTime desc | take 10"

RESPONSE = KUSTO_CLIENT.execute(KUSTO_DATABASE, KUSTO_QUERY)

In [ ]:
# convert to data frame or native test
df = dataframe_from_result_table(RESPONSE.primary_results[0])
df

In [ ]:
# dataframe to CSV
df.to_csv(r'Path where you want to store the exported CSV file\File Name.csv', index = False)

In [ ]:
# bulk insert to azure sql database